In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [3]:
from langchain_chroma import Chroma
# 처음 데이터 생성
# vector_store = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     collection_name='chroma-tax',
#     persist_directory='./chroma-tax'
# )

# 데이터 로딩
vector_store = Chroma(
    collection_name='chroma-tax',
    embedding_function=embedding,
    persist_directory='./chroma-tax'
)

retriever = vector_store.as_retriever(search_kwargs={'k':3})


In [4]:
from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langgraph.graph import StateGraph

class AgentState(TypedDict):
    query: str
    context: List[Document]
    answer: str

In [5]:
def retrieve(state: AgentState) -> AgentState:
    """ 
    사용자의 질문에 기반하여 벡터 스토어에서 관련 문서를 검색합니다.
    Args:
        state (AgentState): 사용자의 질문을 포함한 에이전트의 현재 state
    Returns:
        AgentState: 검색된 문서가 추가된 state를 반환합니다.        
    """

    query = state['query']
    docs = retriever.invoke(query)
    return {'context': docs}

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [7]:
from langchain import hub

generate_prompt = hub.pull('rlm/rag-prompt')

def generate(state: AgentState) -> AgentState:
    """ 
    주어진 state를 기반으로 RAG 체인을 사용하여 응답을 생성합니다.
    Args:
        state (AgentState): 사용자의 질문과 문맥을 포함한 에이전트의 현재 state
    Returns:
        AgentState: 생성된 응답을 포함하는 state를 반환합니다.        
    """
    context = state['context']
    query = state['query']

    rag_chain = generate_prompt | llm

    response = rag_chain.invoke({'question': query, 'context': context})

    return {'answer': response}

c:\git_repo\GJ_class\uv_work\.venv\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [8]:
from langchain import hub
from typing import Literal

doc_releveance_prompt = hub.pull("langchain-ai/rag-document-relevance")

def check_doc_relevence(state: AgentState) -> Literal['relevant', 'irrelevant']:
    """ 
    주어진 state를 기반으로 문서의 관련성을 판단합니다.
    Args:
        state (AgentState): 사용자의 질문과 문맥을 포함한 에이전트의 현재 state
    Returns:
        Literal ['relevant', 'irrelevant']: 문서가 관련성이 높으면 'relevant', 그렇치 않으면 'irrelevant' 를 반환합니다.       
    """

    query = state['query']
    context = state['context']

    doc_relevance_chain = doc_releveance_prompt | llm

    response = doc_relevance_chain.invoke({'question': query, 'documents': context})

    if response['Score'] == 1:
        return 'relevant'
    
    return 'irrelevant'


c:\git_repo\GJ_class\uv_work\.venv\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

dictionary = ['사람과 관련된 표현 -> 거주자']

rewrite_prompt = PromptTemplate.from_template(f"""
사용자의 질문을 보고, 우리 사전을 참고해서 사용자의 질문을 변경해 주세요.
사전: {dictionary}
질문: {{query}}
 """)

def rewrite(state: AgentState) -> AgentState:
    """ 
    사용자의 질문을 사전을 고려하여 변경합니다.
    Args:
        state (AgentState): 사용자의 질문을 포함한 에이전트의 현재 state
    Returns:
        AgentState: 변경된 질문을 포함하는 state를 반환합니다. 
    """
    query = state['query']
    rewrite_chain = rewrite_prompt | llm | StrOutputParser()
    response = rewrite_chain.invoke({'query': query})

    return {'query': response}


In [11]:
# 할루시네이션 확인 코드

from langchain_core.output_parsers import StrOutputParser

hallucination_prompt = PromptTemplate.from_template("""
You are a teacher tasked with evaluating whether a student's answer is based on documents or not,
Given documents, which are excerpts from income tax law, and a student's answer;
If the student's answer is based on documents, respond with "not hallucinated",
If the student's answer is not based on documents, respond with "hallucinated".

documents: {documents}
student_answer: {student_answer}
""")

hallucination_llm = ChatOpenAI(model='gpt-4o', temperature=0)

def check_hallucination(state: AgentState) -> Literal['hallucinated', 'not hallucinated']:
    answer = state['answer']
    context = state['context']

    hallucination_chain = hallucination_prompt | hallucination_llm | StrOutputParser()
    response = hallucination_chain.invoke({"documents" : context, "student_answer" : answer})

    print("거짓 여부 : ", response)

    return response

In [ ]:
# 테스트 코드

# query = "연봉 5천만원 거주자의 소득세는 얼마인가요?"
query = "연봉 5천 세금는 얼마?"
context = retriever.invoke(query)

generate_state = {"query" :  query, "context" : context} # 근데 클래스의 변수를 이렇게 넣을 수 있는가? 랭그래프 안쓰고 그냥 함수에 넣을때 -> 일단 잘 작동한다.
answer = generate(generate_state)

print(f"answer : {answer}")

hallucination_state = {"context" : context, "answer" : answer} 
check_hallucination(hallucination_state)

answer : {'answer': AIMessage(content='죄송합니다. 제공된 문서에서는 연봉 5천만 원에 대한 세금 계산에 필요한 구체적인 정보를 찾을 수 없습니다. 세금 계산은 소득 공제, 세율, 기타 개인 상황에 따라 다를 수 있으므로, 정확한 계산을 위해서는 세무 전문가의 조언을 받는 것이 좋습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 3187, 'total_tokens': 3261, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_cbf1785567', 'finish_reason': 'stop', 'logprobs': None}, id='run--ff43dd6b-6c5a-47f7-b4da-e38b0b7b4f05-0', usage_metadata={'input_tokens': 3187, 'output_tokens': 74, 'total_tokens': 3261, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})}
거짓 여부 :  hallucinated


'hallucinated'

In [18]:
from langchain import hub

helpfulness_prompt = hub.pull('langchain-ai/rag-answer-helpfulness')

def check_helpfulness_grader(state: AgentState) -> Literal['helpful', 'unhelpful']:
    """ 
    사용자의 질문에 기반하여 생성된 답변의 유용성을 평가합니다.
    Args:
        state (AgentState): 사용자의 질문을 포함한 에이전트의 현재 state
    Returns:
        Literal ['helpful', 'unhelpful'] : 답변이 유용하다고 판단되면 'helpful', 그렇지 않으면 'unhelpful'을 반환합니다.
    """

    query = state['query']
    answer = state['answer']

    helpfulness_chain = helpfulness_prompt | llm
    response = helpfulness_chain.invoke({"question" : query, "student_answer":answer})

    if response['Score'] == 1:
        print("유용함 : helpful")
        return "helpful"
    else:
        print("유용하지 않음 : unhelpful")
        return "unhelpful"

c:\git_repo\GJ_class\uv_work\.venv\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [15]:
def check_helpfulness(state: AgentState) -> AgentState:
    """ 
    유용성을 확인하는 자리 표시자 함수입니다.
    """
    return state

In [20]:
# check_helpfulness_grader 테스트 코드

query = "연봉 5천만원 거주자의 소득세는 얼마인가요?"
# query = "연봉 5천 세금는 얼마?"
context = retriever.invoke(query)

generate_state = {"query" :  query, "context" : context}
answer = generate(generate_state)

print(f"answer : {answer}")

helpfulness_state = {"query" : query, "answer" : answer} 
check_helpfulness_grader(helpfulness_state)

answer : {'answer': AIMessage(content='연봉 5천만 원인 거주자의 소득세는 624만 원입니다. 이는 5천만 원 이하 구간의 세율 15%를 적용하여 계산된 금액입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 3020, 'total_tokens': 3065, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_1827dd0c55', 'finish_reason': 'stop', 'logprobs': None}, id='run--9f53b5e3-4d90-4a77-8590-06a65abdc376-0', usage_metadata={'input_tokens': 3020, 'output_tokens': 45, 'total_tokens': 3065, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})}
유용함 : helpful


'helpful'

In [26]:
builder = StateGraph(AgentState)

builder.add_node("retrieve", retrieve)
builder.add_node("generate", generate)
builder.add_node('rewrite', rewrite)
builder.add_node("check_helpfulness", check_helpfulness)

In [ ]:
from langgraph.graph import START, END

builder.add_edge(START, 'retrieve')
builder.add_conditional_edges(
    'retrieve',
    check_doc_relevence,
    {
        "relevant" : "generate",
        # "irrelevant" : "rewrite"
        "irrelevant" : END # 그냥 끝내는 코드
    }
    )
builder.add_edge('rewrite', 'retrieve')
builder.add_conditional_edges(
    'generate',
    check_hallucination,
    {
        'hallucinated' : 'generate',
        'not hallucinated' : "check_helpfulness"
    }
)
builder.add_conditional_edges(
    'check_helpfulness',
    check_helpfulness_grader,
    {
        "helpful" : END,
        "unhelpful" : "rewrite"
    }
)

In [28]:
graph = builder.compile()

In [36]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [31]:
initial_state = {"query": "연봉 5천만원 직장인의 소득세는?"}
response = graph.invoke(initial_state)

거짓 여부 :  not hallucinated
유용함 : helpful


In [32]:
response

{'query': '거주자의 연봉 5천만원일 경우 소득세는?',
 'context': [Document(metadata={'source': './data/tax.docx'}, page_content='③ 제1항 및 제2항에 따른 이자등 상당액의 계산방법과 그 밖에 필요한 사항은 대통령령으로 정한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제46조(채권 등에 대한 소득금액의 계산 특례) ① 거주자가 제16조제1항제1호ㆍ제2호ㆍ제2호의2ㆍ제5호 및 제6호에 해당하는 채권 또는 증권과 타인에게 양도가 가능한 증권으로서 대통령령으로 정하는 것(이하 이 조, 제133조의2 및 제156조의3에서 “채권등”이라 한다)의 발행법인으로부터 해당 채권등에서 발생하는 이자 또는 할인액(이하 이 조, 제133조의2 및 제156조의3에서 “이자등”이라 한다)을 지급[전환사채의 주식전환, 교환사채의 주식교환 및 신주인수권부사채의 신주인수권행사(신주 발행대금을 해당 신주인수권부사채로 납입하는 경우만 해당한다) 및 「자본시장과 금융투자업에 관한 법률」 제4조제7항제3호ㆍ제3호의2 및 제3호의3에 해당하는 채권등이 주식으로 전환ㆍ상환되는 경우를 포함한다. 이하 같다]받거나 해당 채권등을 매도(증여ㆍ변제 및 출자 등으로 채권등의 소유권 또는 이자소득의 수급권의 변동이 있는 경우와 매도를 위탁하거나 중개 또는 알선시키는 경우를 포함하되, 환매조건부채권매매거래 등 대통령령으로 정하는 경우는 제외한다. 이하 제133조의2에서 같다)하는 경우에는 거주자에게 그 보유기간별로 귀속되는 이자등 상당액을 해당 거주자의 제16조에 따른 이자소득으로 보아 소득금액을 계산한다. <개정 2010. 12. 27., 2012. 1. 1., 2020. 12. 29.>\n\n② 제1항을 적용할 때 해당 거주자가 해당 채권등을 보유한 기간을 대통령령으로 정하는 바에 따라 입증하지 못하는 경우에는 제133조의2제1항에 따른 원천징수기간의 이자등 상당액이 해당 거주자에게 귀속되는 것으로 보아 소득금액을 계산한

In [33]:
print(response['answer'].content)

거주자의 연봉이 5천만 원인 경우, 소득세는 624만 원입니다. 이는 종합소득 과세표준이 5천만 원 이하일 때 적용되는 세율에 따른 계산 결과입니다.
